<b>Note: This Jupyter Notebook is associated with the article [How to Trade BTC/USD on Alpaca and TradingView](https://alpaca.markets/learn/btc-usd-alpaca-tradingview).</b>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/crypto/crypto-btc-usd-swing-trade.ipynb)

# Step 1: Setting Up the Bitcoin Trading Environment

In [2]:
# Install or upgrade the package `alpaca-py` and import it
# !python3 -m pip install --upgrade alpaca-py
# Install or upgrade the package `pandas-ta`
# !python3 -m pip install -U git+https://github.com/twopirllc/pandas-ta
!python3 -m pip install --upgrade alpaca-py
!python3 -m pip install -U git+https://github.com/twopirllc/pandas-ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 3.3 MB/s eta 0:00:00
  Cloning https://github.com/twopirllc/pandas-ta to /tmp/pip-req-build-z7zz43m0
  Running command git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-z7zz43m0
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-z7zz43m0 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-z7zz43m0 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [5]:
pip install pandas-ta

In [4]:
# Import standard library modules
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# Import third-party modules
import numpy as np
import pandas as pd
import pandas_ta as ta  # The installment of pandas-ta may be tricky. Please find the pypi page here (https://pypi.org/project/pandas-ta/)
import matplotlib.pyplot as plt

# Import Alpaca modules
import alpaca
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.data.requests import (
    CryptoBarsRequest,
    CryptoLatestQuoteRequest,
    CryptoQuoteRequest,
    CryptoTradesRequest,
)
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import (
    AssetClass,
    AssetStatus,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)
from alpaca.trading.requests import LimitOrderRequest, MarketOrderRequest

ImportError: cannot import name 'NaN' from 'numpy' (/usr/local/lib/python3.11/dist-packages/numpy/__init__.py)

In [ ]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

API_KEY = "<YOUR PAPER API KEY>"
API_SECRET = "<YOUR PAPER API SECRET>"

#### We use paper environment for this example ####
PAPER = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

# setup trading clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=trade_api_url)
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Step 2: Executing Market Data Analysis

In [ ]:
# we will place orders which Alapca trading platform supports
# - order types for crypto: market, limit, stop_limit
# - time_in_force for crypto: gtc, ioc.
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder

# we will place orders for symbol: BTC/USD in this example
symbol = "BTC/USD"

In [ ]:
# Set the timezone
timezone = ZoneInfo("America/New_York")

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define the start and end dates for the historical period
start_date = today - timedelta(days=365 * 2)
end_date = today - timedelta(days=1)

# Create the request object
req = CryptoBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame.Day,
    start=start_date,
    end_date=end_date,
)

bars_data = crypto_historical_data_client.get_crypto_bars(req).df.reset_index(level='symbol', drop=True)

In [ ]:
# Create a copy of the dataframe to avoid modifying the original data.
df = bars_data.copy()
df.head()

## Calculating EMA

In [ ]:
# Define Exponential Moving Average (EMA) periods
short_window = 12 # Common short-term period
long_window = 26  # Common long-term period

# Calculate EMA using pandas.ewm()
# 'span' is related to the period N, often approximated as span = N for typical EMA calculations.
# 'adjust=False' uses a standard recursive formula common in technical analysis.
df["ema_short"] = df["close"].ewm(span=short_window, adjust=False).mean()
df["ema_long"] = df["close"].ewm(span=long_window, adjust=False).mean()


## Calculating SMA, ATR, and ADX

In [ ]:
# Calculate Simple Moving Average (SMA) using rolling() with 50-day SMA
sma_window = 50
df["sma"] = df["close"].rolling(window=sma_window).mean()
df[["close", "sma"]].tail()

In [ ]:
# Calculate Average True Range (ATR) - requires high, low, close columns
# Default length is 14 periods
df.ta.atr(length=14, append=True) # Appends 'ATR_14' column

# Calculate Average Directional Index (ADX) - requires high, low, close
# Default length is 14 periods
# Note: ta.adx() returns ADX, Positive Directional Movement (DMP), and Negative Directional Movement (DMN)
df.ta.adx(length=14, append=True) # Appends 'ADX_14', 'DMP_14', 'DMN_14' columns

## Visualizing Indicators

In [ ]:
if not df.empty and "ema_short" in df.columns and "ema_long" in df.columns:
    plt.style.use("seaborn-v0_8-darkgrid") # Use a visually appealing style
    plt.figure(figsize=(14, 7))

    plt.plot(df.index, df["close"], label="Close Price", color="blue", alpha=0.7)
    plt.plot(df.index, df["ema_short"], label=f"EMA ({short_window})", color="orange", linestyle="--")
    plt.plot(df.index, df["ema_long"], label=f"EMA ({long_window})", color="purple", linestyle=":")

    # Optional: Add crossover points visually
    buy_signals = df[df["buy_signal"]] if "buy_signal" in df else pd.DataFrame()
    sell_signals = df[df["sell_signal"]] if "sell_signal" in df else pd.DataFrame()

    if not buy_signals.empty:
        plt.scatter(buy_signals.index, buy_signals["close"], label="Buy Signal", marker="^", color="green", s=100, zorder=5)
    if not sell_signals.empty:
        plt.scatter(sell_signals.index, sell_signals["close"], label="Sell Signal", marker="v", color="red", s=100, zorder=5)

    plt.title(f'{symbol} Price and EMA Crossover Signals')
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.tight_layout() # Adjust layout to prevent labels overlapping
    plt.show()
else:
    print("Cannot plot: DataFrame is empty or EMA columns are missing.")

# Step 3: Creating a Bitcoin Swing Trading Algorithm with EMAs

In [ ]:
# Generate Buy Signal
# Condition 1: Is the short EMA currently above the long EMA?
currently_above = df["ema_short"] > df["ema_long"]
# Condition 2: Was the short EMA below or equal to the long EMA on the previous bar?
previously_below_or_equal = df["ema_short"].shift(1) <= df["ema_long"].shift(1)
# Buy Signal: True only on the bar where the crossover *just* happened
df["buy_signal"] = currently_above & previously_below_or_equal

# Generate Sell Signal
# Condition 1: Is the short EMA currently below the long EMA?
currently_below = df["ema_short"] < df["ema_long"]
# Condition 2: Was the short EMA above or equal to the long EMA on the previous bar?
previously_above_or_equal = df["ema_short"].shift(1) >= df["ema_long"].shift(1)
# Sell Signal: True only on the bar where the crossover *just* happened
df["sell_signal"] = currently_below & previously_above_or_equal

# Display rows where signals occurred
print("\nCrossover Signal Occurrences:")
signal_rows = df[df["buy_signal"] | df["sell_signal"]]
if not signal_rows.empty:
    print(signal_rows[["close", "ema_short", "ema_long", "buy_signal", "sell_signal"]])
else:
    print("No crossover signals found in the data.")


In [ ]:
# Store the latest signal information for potential execution
latest_data = df.iloc[-1]
print("\nLatest Data Row:")
latest_data[['close', 'ema_short', 'ema_long', 'buy_signal', 'sell_signal']]

# Step 4: Purchasing BTC/USD using Python with Alpaca's Trading API

## Submitting Simple Market Orders

In [ ]:
# simple, market order, notional
req = MarketOrderRequest(
    symbol = symbol,
    notional = 250,  # notional is specified in USD, here we specify $250
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

## Submitting Limit Order Requests

In [ ]:
# Function to get latest quotes by symbol
def get_latest_quote(symbol):
    req = CryptoLatestQuoteRequest(symbol_or_symbols = [symbol])
    res = crypto_historical_data_client.get_crypto_latest_quote(req)
    return res

# Get latest quotes
btc_quote = get_latest_quote("BTC/USD")

In [ ]:
# Get the estimated buying cost of BTC/USD with given qty
def calc_cost(quote, qty):
    # Calculate total cost of BTC/USD order and round to 4 decimal places
    total_cost = round(qty * quote[symbol].ask_price, 4)
    return total_cost

In [ ]:
QTY = 0.003
cost = calc_cost(btc_quote, qty=QTY)
f"The total cost of BTC/USD order for qtry={QTY} is ${cost}"

In [ ]:
# Define a 6% range around the crypto price
LIMIT_RANGE = 0.06

# Calculate the limit price as 6% below the current ask price
limit_price = btc_quote[symbol].ask_price * (1 - LIMIT_RANGE)
f"The limit price for BTC/USD 'BUY' order for qty={QTY} is ${limit_price}"

In [ ]:
# simple, market order, notional
req = LimitOrderRequest(
    symbol=symbol,
    qty=QTY,
    limit_price = limit_price,
    side=OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force=TimeInForce.GTC, # TIF=DAY is not available
)

res = trade_client.submit_order(req)
res